In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [2]:
import numpy as np
import pandas as pd
import sys
import random

from tqdm import tqdm

import string

import os

import json

In [3]:
!pip install -q transformers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 895 kB 69.5 MB/s 
     |████████████████████████████████| 3.3 MB 44.5 MB/s 
     |████████████████████████████████| 596 kB 54.5 MB/s 


In [4]:
from transformers import AutoTokenizer, BertConfig, TFBertModel

In [5]:
import tensorflow as tf
from tensorflow.keras.losses import sparse_categorical_crossentropy as sce
from tensorflow.keras.callbacks import Callback

In [6]:
train_ins = '/content/drive/MyDrive/data/fine_data/train_instance_192.csv'
val_ins = '/content/drive/MyDrive/data/fine_data/val_instance_192.csv'

val_id = '/content/drive/MyDrive/data/fine_data/val_id.csv'
train_id = '/content/drive/MyDrive/data/fine_data/train_id.csv'

val_cand = '/content/drive/MyDrive/data/fine_data/val_candidates.csv'
gt = '/content/drive/MyDrive/data/fine_data/validation.csv'

## **Functions to tokenize data**

In [7]:
model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)

tags = ['<Dd>', '<Dl>', '<Dt>', '<H1>', '<H2>', '<H3>', '<Li>', '<Ol>', '<P>', '<Table>', '<Td>', '<Th>', '<Tr>', '<Ul>',
        '</Dd>', '</Dl>', '</Dt>', '</H1>', '</H2>', '</H3>', '</Li>', '</Ol>', '</P>', '</Table>', '</Td>', '</Th>', '</Tr>', '</Ul>',
        '<Th_colspan=', '</Th_colspan=', '``', '\'\'', '--']

# for i in range(0, 51, 1):
#     tags.append(f'[part={i}]')

print(tags)

special_tokens_dict = {'additional_special_tokens': tags}

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(num_added_toks)
print(len(tokenizer))

# encoder.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

['<Dd>', '<Dl>', '<Dt>', '<H1>', '<H2>', '<H3>', '<Li>', '<Ol>', '<P>', '<Table>', '<Td>', '<Th>', '<Tr>', '<Ul>', '</Dd>', '</Dl>', '</Dt>', '</H1>', '</H2>', '</H3>', '</Li>', '</Ol>', '</P>', '</Table>', '</Td>', '</Th>', '</Tr>', '</Ul>', '<Th_colspan=', '</Th_colspan=', '``', "''", '--']
33
30555


In [8]:
AnswerType = {
    'NO_ANSWER': 0,
    'YES': 1,
    'NO': 2,
    'SHORT' : 3,
    'LONG' : 4
}

def preprocess_data(data, tokenizer, debug=False): 
    progress = tqdm(data, total=len(data))
    x1 = []
    x2 = []
    x3 = []
    y = []
    for sam in progress:
        # part_id = sam['part_id']
        # part_tokens = f'[part={part_id}]'

        # context = part_tokens + " " + sam['context']
        tokenized_sam = tokenizer.encode_plus(sam['question'], sam['context'], 
                                              padding='max_length',
                                              truncation=True,
                                              max_length=512,
                                              add_special_tokens=True)
        
        x1.append(tf.cast(tokenized_sam['input_ids'], tf.int32))
        x2.append(tf.cast(tokenized_sam['token_type_ids'], tf.int32))
        x3.append(tf.cast(tokenized_sam['attention_mask'], tf.int32))

        y.append([sam['start'], sam['stop'], AnswerType[sam['target']]])

    x1 = tf.convert_to_tensor(x1)
    x2 = tf.convert_to_tensor(x2)
    x3 = tf.convert_to_tensor(x3)

    y = tf.convert_to_tensor(y)
    return x1, x2, x3, y


In [9]:
train_ins_df = pd.read_csv(train_ins)
tran_ins_list = train_ins_df.to_dict('records')
print(len(tran_ins_list))

432850


In [10]:
val_id_df = pd.read_csv(val_id)
val_id_list = val_id_df['example_id'].tolist()

val_cand_df = pd.read_pickle(val_cand)
val_cand_list = val_cand_df.to_dict('records') 

gt_df = pd.read_csv(gt)
gt_list = gt_df.to_dict('records')

In [11]:
val_ins_df = pd.read_csv(val_ins)
val_ins_list = val_ins_df.to_dict('records')
print(len(val_ins_list))

142202


In [12]:
def get_strategy():
    try:
        tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
        print('Running on TPU ', tpu_cluster_resolver.cluster_spec().as_dict()['worker'])
        tf.config.experimental_connect_to_cluster(tpu_cluster_resolver)
        tf.tpu.experimental.initialize_tpu_system(tpu_cluster_resolver)
        strategy = tf.distribute.experimental.TPUStrategy(tpu_cluster_resolver)
    except ValueError as e:
        print(e)
        print('No TPU detected')
        tpu = None
        strategy = tf.distribute.get_strategy()
    return strategy

In [13]:
def create_model(tokenizer, model_name, debug=False):
    config = BertConfig()
    if debug:
        print(config)
    # encoder = TFBertModel(config)
    encoder = TFBertModel.from_pretrained(model_name)
    encoder.resize_token_embeddings(len(tokenizer))

    NUM_TARGET = 5
    class MyQAModel(tf.keras.Model):
        def __init__(self, *inputs, **kwargs):
            super().__init__(*inputs, **kwargs)            
            self.bert = encoder

            # self.dropout_start = tf.keras.layers.Dropout(0.1)
            # self.dropout_stop = tf.keras.layers.Dropout(0.1)
            # self.dropout_target = tf.keras.layers.Dropout(0.1)

            self.start_logits = tf.keras.layers.Dense(1)
            self.stop_logits = tf.keras.layers.Dense(1)
            
            self.target = tf.keras.layers.Dense(NUM_TARGET)

        def call(self, inputs, **kwargs):
            bert_res=self.bert(inputs[0], 
                               token_type_ids=inputs[1], 
                               attention_mask=inputs[2]
                               )
            
            # dropout_res1 = self.dropout_start(bert_res[0])
            dropout_res1 = bert_res[0]

            start_logits = tf.squeeze(self.start_logits(dropout_res1), -1)

            # dropout_res2 = self.dropout_stop(bert_res[0])
            dropout_res2 = bert_res[0]

            stop_logits = tf.squeeze(self.stop_logits(dropout_res2), -1)

            # dropout_res3 = self.dropout_target(bert_res[1])
            dropout_res3 = bert_res[1]
            
            targets = self.target(dropout_res3)
            
            paddings = tf.constant([[0, 0,], [0, 512-NUM_TARGET]])
            targets = tf.pad(targets, paddings)
            
            res = tf.stack([start_logits, stop_logits, targets], axis=1)
            return res
        
    model = MyQAModel()
    return model 

In [14]:
def getFineInsRes(raw_res, ins_list, debug=False):
    list_ins_res = []
    progress = tqdm(ins_list, total=len(ins_list))
    for rowid, row in enumerate(progress):
        example_id = row['example_id']
        part_id = row['part_id']

        res_start = tf.nn.softmax(raw_res[rowid][0]).numpy()
        res_stop = tf.nn.softmax(raw_res[rowid][1]).numpy()
        res_target = tf.nn.softmax(raw_res[rowid][2]).numpy()
        
        start = np.argmax(res_start)
        stop = np.argmax(res_stop)
        target = np.argmax(res_target)

        start_score = res_start[start]
        stop_score = res_stop[stop]

        start_CLS = res_start[0]
        stop_CLS = res_stop[0]
        
        ins_res = {}
        ins_res['example_id'] = example_id
        ins_res['part_id'] = part_id

        ins_res['start'] = start 
        ins_res['stop'] = stop
        ins_res['target'] = target

        ins_res['start_score'] = start_score
        ins_res['stop_score'] = stop_score
        
        ins_res['start_CLS'] = start_CLS
        ins_res['stop_CLS'] = stop_CLS
        if debug:
            if rowid == 101:
                print(row)
                print(ins_res)
        list_ins_res.append(ins_res)
    list_ins_res_df = pd.DataFrame(list_ins_res)
    return list_ins_res_df

In [15]:
def mergeDocumentRes(ins_df, val_id_df, threshold=0.0001, debug=False):
    STRIDE = 192
    list_doc_lan = []
    for idx, doc in val_id_df.iterrows():
        doc_id = doc['example_id']
        ins_of_doc = ins_df.loc[ins_df['example_id'] == doc_id]
        
        start_ins = ins_of_doc.loc[ins_of_doc['start'] != 0]
        stop_ins = ins_of_doc.loc[ins_of_doc['stop'] != 0]
        all_non_zero = pd.concat([start_ins,stop_ins]).drop_duplicates()
        
        best_start = -1
        best_stop = -1
        best_target = 0
        best_score = threshold
                    
        for idx_ins, ins in all_non_zero.iterrows():
            ins_start = int(ins['start'])
            ins_stop = int(ins['stop'])
            ins_target = int(ins['target'])
            
            part_id = ins['part_id']
            part_start = part_id*STRIDE
            
            real_start = int(ins_start + part_start)
            real_stop = int(ins_stop + part_start)
            
            s_start = ins['start_score']
            s_stop = ins['stop_score']
            
            cls_start = ins['start_CLS']
            cls_stop = ins['stop_CLS']
            
            if real_stop > real_start:   
                if s_start - cls_start + s_stop - cls_stop > best_score:
                    best_score = s_start - cls_start + s_stop - cls_stop
                    best_start = real_start
                    best_stop = real_stop
                    best_target = ins_target

        doc_lan = {}
        doc_lan['example_id'] = doc_id
        doc_lan['start'] = best_start
        doc_lan['stop'] = best_stop
        doc_lan['target'] = best_target
        doc_lan['score'] = best_score
        
        if debug:
            if idx == 101:
                print(doc_lan)
        
        list_doc_lan.append(doc_lan)
    
    list_doc_lan_df = pd.DataFrame(list_doc_lan)
    return list_doc_lan_df

In [16]:
AnswerTypeRev = {
    0: 'NO_ANSWER',
    1: 'YES',
    2: 'NO',
    3: 'SHORT',
    4: 'LONG'
}

def getSubmission(doc_res_df, doc_cand_df, threshold=0.0001, debug=False):
    doc_res_df.example_id = doc_res_df.example_id.astype(str)
    doc_cand_df.example_id = doc_cand_df.example_id.astype(str)
    if debug:
        print(doc_res_df.dtypes)
        print(doc_cand_df.dtypes)

    combine_df = pd.merge(doc_res_df, doc_cand_df, on='example_id')
    lines = []
    for id, doc in combine_df.iterrows():

        example_id = doc['example_id']
        long_id = str(example_id) + '_long'
        short_id = str(example_id) + '_short'

        line_long = {}
        line_short = {}
        line_long['example_id'] = long_id
        line_short['example_id'] = short_id

        an_start = int(doc['start'])
        an_stop = int(doc['stop'])
        an_target = doc['target']
        an_score = doc['score']
        # print(an_start, an_stop, an_target, an_score)
        lan_start = -1
        lan_stop = -1
        san_start = -1
        san_stop = -1

        # find long answer 
        if an_start > 0 and an_stop > 0:
            candidates = doc['list_candidates']
            an_range = [*range(an_start, an_stop + 1, 1)]
            # print(an_range)

            best_inter = 0.5
            shortest = 10000000000000
            best_id = 0
            # print(candidates)
            for cidx, cand in enumerate(candidates):
                # print(cand)
                c_start = int(cand['start_token'])
                c_stop = int(cand['end_token'])

                c_range = [*range(c_start, c_stop + 1, 1)]
                inter = len(list(set(an_range)&set(c_range)))
            
                if float(inter) > best_inter:
                    best_id = cidx
                    best_inter = inter
                    shortest = len(c_range)
                elif inter == best_inter:
                    if shortest > len(c_range):
                        best_id = cidx
                        shortest = len(c_range)


            lan_start = candidates[best_id]['start_token']
            lan_stop = candidates[best_id]['end_token']
            san_start = an_start
            san_stop = an_stop
            an_target = AnswerTypeRev[an_target]

            if debug:
                if id == 101:
                    print(lan_start, lan_stop, san_start, san_stop, an_target)

        if lan_start > 0 and lan_stop > 0:
            long_string = str(lan_start) + ':' + str(lan_stop)
        else:
            long_string = ''

        if san_start > 0 and san_stop > 0:
            short_string = str(san_start) + ":" + str(san_stop)
        else:
            short_string = ''
        
        if an_target == 'YES' or an_target == "NO":
            short_string = an_target

        elif an_target == 'LONG':
            short_string = ''

        elif float(san_stop - san_start) > float(lan_stop - lan_start)*0.9:
            short_string = ''

        line_long['PredictionString'] = long_string
        line_short['PredictionString'] = short_string
        lines.append(line_long)
        lines.append(line_short)

    lines_df = pd.DataFrame(lines)
    sorted_df = lines_df.sort_values('example_id')
    return sorted_df

In [17]:
def getResult(gt_df, res_df, debug=False):
    gt_df.fillna('', inplace=True)
    if gt_df.shape[0] != res_df.shape[0]:
        print('ERROR: Different number of rows')
        return -1.0
    
    TP, TN, FP, FN = 0, 0, 0, 0

    # list_incorrect = []

    gt_id = gt_df['example_id'].astype(str).tolist()
    res_id = res_df['example_id'].astype(str).tolist()

    if gt_id != res_id:
        print("ERROR: Example_id lists are not the same")
        return -1.0
    
    gt_res = gt_df['PredictionString'].tolist()
    res_res = res_df['PredictionString'].tolist()
    id_list = gt_df['example_id'].tolist()
    for i in range(len(gt_res)):
        if gt_res[i] == res_res[i]:
            if gt_res[i] != "":
                TP += 1
            else:
                TN += 1
        else:
            if res_res[i] == '':
                FN += 1
            else:
                FP += 1
    recall = float(TP)/float(TP+FN + 0.000001)
    precision = float(TP)/float(TP+FP + 0.000001)
    print("TP: ", TP)
    print("TN: ", TN)
    print("FP: ", FP)
    print("FN: ", FN)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("F1: ", float(2*recall*precision)/float(recall + precision + 0.00000001))


In [18]:
class IntervalEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.x_val, self.y_val = validation_data
        self.list_threshold = [0.0001, 0.001, 0.01, 0.1, 0.15, 0.2]

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.x_val, verbose=1)
            
            fineResDf = getFineInsRes(y_pred, val_ins_list)

            for threshold in self.list_threshold:
                print('THRESHOLD: ', threshold)
                docAnsDf = mergeDocumentRes(fineResDf, val_id_df, threshold=threshold)
                sub = getSubmission(docAnsDf, val_cand_df)
                getResult(gt_df, sub)

In [19]:
x1, x2, x3, y = preprocess_data(tran_ins_list, tokenizer, debug=True)

100%|██████████| 432850/432850 [24:13<00:00, 297.83it/s]


In [20]:
x_val1, x_val2, x_val3, y_val = preprocess_data(val_ins_list, tokenizer)

100%|██████████| 142202/142202 [07:50<00:00, 302.45it/s]


In [21]:
strategy = get_strategy()
# weight_path = '/content/drive/MyDrive/data/models/first-train-batch=128-lr=0.00005/weights-05.h5'
weight_path=''
with strategy.scope():
    myQAModel = create_model(tokenizer, model_name)

    if os.path.isfile(weight_path):
        myQAModel.predict([x1[0:1], x2[0:1], x3[0:1]])
        myQAModel.load_weights(weight_path)

    opt = tf.keras.optimizers.Adam(learning_rate=0.00005)
    lossSCE = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metricSCA = tf.keras.metrics.SparseCategoricalAccuracy()
    myQAModel.compile(optimizer=opt, loss=lossSCE, metrics=[metricSCA], run_eagerly=False)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.51.128.218:8470']
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.51.128.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.51.128.218:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
DEBUG:filelock:Attempting to acquire lock 140045372837840 on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock
DEBUG:filelock:Lock 140045372837840 acquired on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140045372837840 on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock
DEBUG:filelock:Lock 140045372837840 released on /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5.lock
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification mo

In [22]:
save_locally = tf.train.CheckpointOptions(experimental_io_device='/job:localhost')

filepath="/content/drive/MyDrive/data/models/current/weights-{epoch:02d}.h5"
checkpoint_dir = os.path.dirname(filepath)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                                verbose=1, 
                                                save_best_only=False, 
                                                save_weights_only=True,
                                                options=save_locally)
eval = IntervalEvaluation(validation_data=([x_val1, x_val2, x_val3], y_val))

In [23]:
history = myQAModel.fit(x=[x1, x2, x3],
                        y=y,
                        batch_size=128,
                        callbacks=[checkpoint, eval], 
                        epochs=100, 
                        validation_data=([x_val1, x_val2, x_val3], y_val), 
                        validation_batch_size=128)

Epoch 1/100


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 3) dtype=int32>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 3) dtype=int32>]


   6/3382 [..............................] - ETA: 25:39 - loss: 5.3490 - sparse_categorical_accuracy: 0.2713WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0090s vs `on_train_batch_end` time: 9.9209s). Check your callbacks.


3382/3382 [==============================] - ETA: 0s - loss: 2.5018 - sparse_categorical_accuracy: 0.4793

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 3) dtype=int32>]


...

100%|██████████| 142202/142202 [10:10<00:00, 233.03it/s]


THRESHOLD:  0.0001
TP:  700
TN:  984
FP:  3994
FN:  472
Recall:  0.597269624063763
Precision:  0.14912654449315582
F1:  0.23866348120795736
THRESHOLD:  0.001
TP:  700
TN:  984
FP:  3994
FN:  472
Recall:  0.597269624063763
Precision:  0.14912654449315582
F1:  0.23866348120795736
THRESHOLD:  0.01
TP:  700
TN:  984
FP:  3994
FN:  472
Recall:  0.597269624063763
Precision:  0.14912654449315582
F1:  0.23866348120795736
THRESHOLD:  0.1
TP:  700
TN:  1009
FP:  3960
FN:  481
Recall:  0.592718035061204
Precision:  0.1502145922424432
F1:  0.23968498213942668
THRESHOLD:  0.15
TP:  699
TN:  1015
FP:  3950
FN:  486
Recall:  0.5898734172237354
Precision:  0.15035491500315015
F1:  0.23962975327985533
THRESHOLD:  0.2
TP:  695
TN:  1030
FP:  3930
FN:  495
Recall:  0.5840336129545936
Precision:  0.1502702702377794
F1:  0.23903697000729723
Epoch 6/100
3382/3382 [==============================] - 1708s 505ms/step - loss: 1.0983 - sparse_categorical_accuracy: 0.6610 - val_loss: 0.5692 - val_sparse_categoric

100%|██████████| 142202/142202 [09:59<00:00, 237.28it/s]


THRESHOLD:  0.0001
TP:  672
TN:  934
FP:  4101
FN:  443
Recall:  0.6026905824191116
Precision:  0.14079195471594552
F1:  0.22826086641752533
THRESHOLD:  0.001
TP:  672
TN:  934
FP:  4101
FN:  443
Recall:  0.6026905824191116
Precision:  0.14079195471594552
F1:  0.22826086641752533
THRESHOLD:  0.01
TP:  672
TN:  936
FP:  4099
FN:  443
Recall:  0.6026905824191116
Precision:  0.14085097460891824
F1:  0.2283384270247484
THRESHOLD:  0.1
TP:  671
TN:  952
FP:  4077
FN:  450
Recall:  0.598572702409837
Precision:  0.14132266214378209
F1:  0.22865905288890478
THRESHOLD:  0.15
TP:  670
TN:  975
FP:  4051
FN:  454
Recall:  0.5960854087223438
Precision:  0.1419190849095702
F1:  0.22925577098108085
THRESHOLD:  0.2
TP:  668
TN:  991
FP:  4028
FN:  463
Recall:  0.5906277625193388
Precision:  0.14224872228657395
F1:  0.22927749807995654
Epoch 7/100
   8/3382 [..............................] - ETA: 25:39 - loss: 1.0198 - sparse_categorical_accuracy: 0.6699

KeyboardInterrupt: ignored